<a href="https://colab.research.google.com/github/bsyh/foursquare/blob/main/Nltk_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [4]:
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 34.0 MB/s 
     |████████████████████████████████| 2.0 MB 59.3 MB/s 
     |████████████████████████████████| 102 kB 53.6 MB/s 


In [5]:
from google.colab import drive

drive.mount('/content/drive')
root_path = 'drive/MyDrive/Foursquare/'
!ls

Mounted at /content/drive
drive  sample_data


In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer

import Levenshtein as lev
import math
from collections import Counter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    return summary

In [8]:
!ls

drive  sample_data


In [9]:
file_loc = "/content/drive/MyDrive/Foursquare/data/pairs_tiny_800.csv"

In [10]:
df = pd.read_csv(file_loc)

In [11]:
df.head()

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,...,3.635206,NaN,NaN,NaN,NaN,BE,NaN,NaN,Bars,True
1,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,...,27.845560,batı aydın,aydın,NaN,67500,TR,NaN,NaN,Electronics Stores,False
2,E_000023d8f4be44,Island Spa,14.518970,121.018702,"5th Flr, Newport Mall, Resorts World Manila",Pasay City,Metro Manila,NaN,PH,NaN,...,121.019827,NaN,NaN,NaN,NaN,PH,NaN,NaN,Spas,True
3,E_00007dcd2bb53f,TOGO'S Sandwiches,38.257797,-122.064599,"1380 Holiday Ln., Ste. B",Fairfield,CA,94534,US,https://locations.togos.com/ll/US/CA/Fairfield...,...,-122.064606,NaN,Fairfield,CA,NaN,US,NaN,NaN,Sandwich Places,True
4,E_0000c362229d93,Coffee Cat,7.082218,125.610244,F. Torres St.,Davao City,Davao Region,8000,PH,NaN,...,125.611457,E. Jacinto Extension,Davao City,Davao Region,8000,PH,NaN,NaN,"Coffee Shops, Cafés, Dessert Shops",False


In [12]:
print ("Total number of rows in dataset = {}".format(df.shape[0]))
print ("Total number of columns in dataset = {}".format(df.shape[1]))

Total number of rows in dataset = 800
Total number of columns in dataset = 25


In [13]:
result = resumetable(df)
result

Dataset Shape: (800, 25)


,Name,dtypes,Missing,Uniques,First Value,Second Value
0,id_1,object,0,667,E_000001272c6c5d,E_000008a8ba4f48
1,name_1,object,0,654,Café Stad Oudenaarde,Turkcell
2,latitude_1,float64,0,665,50.859975,37.84451
3,longitude_1,float64,0,665,3.634196,27.844202
4,address_1,object,152,531,Abdijstraat,Adnan Menderes Bulvarı
5,city_1,object,90,485,Nederename,NaN
6,state_1,object,176,273,Oost-Vlaanderen,NaN
7,zip_1,object,295,408,9700,NaN
8,country_1,object,0,65,BE,TR
9,url_1,object,468,256,NaN,NaN


In [16]:
# 2 word preprocessing

# lowercase
def lower(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: x.lower())
    return df

# number removing
def num_remove(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: re.sub(r'\d+', '', x))
    return df

# punctuation removal
def punc_remove(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: x.translate(str.maketrans("","", string.punctuation)))
    return df

# white spaces removal
def space_remove(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: x.strip())
    return df

# stop words removal

def list_to_string(lis):
    string = ''
    for i in lis:
        string += i
        string += ' '
    return string

def stop(string):
    stops = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    result = [i for i in tokens if not i in stops]
    return result
    
def stop_remove(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: stop(x))
    return df

# stemming and lemmatization
def lemma_stem(lis):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    new_lis = []
    for word in lis:
        word_new = lemmatizer.lemmatize(word)
        word_new = stemmer.stem(word_new)
        new_lis += [word_new]
    return new_lis

def lemma_stemming(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: lemma_stem(x))
    return df

# remove url
def remove_URL(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: re.sub(r"http\S+", "", x))
    return df

In [17]:
pairs = pd.read_csv('/content/drive/MyDrive/Foursquare/data/pairs_tiny_100.csv')
pairs = pairs
pairs

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,...,3.635206,NaN,NaN,NaN,NaN,BE,NaN,NaN,Bars,True
1,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,...,27.845560,batı aydın,aydın,NaN,67500,TR,NaN,NaN,Electronics Stores,False
2,E_000023d8f4be44,Island Spa,14.518970,121.018702,"5th Flr, Newport Mall, Resorts World Manila",Pasay City,Metro Manila,NaN,PH,NaN,...,121.019827,NaN,NaN,NaN,NaN,PH,NaN,NaN,Spas,True
3,E_00007dcd2bb53f,TOGO'S Sandwiches,38.257797,-122.064599,"1380 Holiday Ln., Ste. B",Fairfield,CA,94534,US,https://locations.togos.com/ll/US/CA/Fairfield...,...,-122.064606,NaN,Fairfield,CA,NaN,US,NaN,NaN,Sandwich Places,True
4,E_0000c362229d93,Coffee Cat,7.082218,125.610244,F. Torres St.,Davao City,Davao Region,8000,PH,NaN,...,125.611457,E. Jacinto Extension,Davao City,Davao Region,8000,PH,NaN,NaN,"Coffee Shops, Cafés, Dessert Shops",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,E_000d7da93f373c,ローソン 小樽色内店,43.198794,140.998369,色内1-15-13,小樽市,北海道,047-0031,JP,https://www.lawson.co.jp,...,140.998318,色内1-15-13,小樽,北海道,047-0031,JP,https://www.lawson.co.jp,NaN,Convenience Stores,True
96,E_000da284c9870c,Cityplace Parade 2,43.640253,-79.397529,21 Iceboat Terrace & 151 Dan Leckie Way,Toronto,ON,M5V 4A9,CA,http://www.cityplace.ca,...,-79.398128,21 Iceboat Terrace,Toronto,ON,M5V 4A9,CA,NaN,NaN,Residential Buildings (Apartments / Condos),True
97,E_000dfeb9eb4e87,Plac Zawiszy,52.225121,20.989843,NaN,Warszawa,Mazowieckie,02-017,PL,NaN,...,20.987290,NaN,Warsaw,Masovian,00-811,PL,NaN,NaN,Bus Stops,False
98,E_000e3b02cbde30,Walmart Neighborhood Market,36.055210,-115.119751,7445 S Eastern Ave,Las Vegas,NV,89123,US,https://www.walmart.com/store/3356/las-vegas-n...,...,-115.136438,NaN,Las Vegas,NV,89119,US,NaN,NaN,Miscellaneous Shops,True


In [18]:
pairs.iloc[:,0:12]
pairs.info()
pairs.match.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_1          100 non-null    object 
 1   name_1        100 non-null    object 
 2   latitude_1    100 non-null    float64
 3   longitude_1   100 non-null    float64
 4   address_1     85 non-null     object 
 5   city_1        89 non-null     object 
 6   state_1       78 non-null     object 
 7   zip_1         68 non-null     object 
 8   country_1     100 non-null    object 
 9   url_1         47 non-null     object 
 10  phone_1       48 non-null     object 
 11  categories_1  100 non-null    object 
 12  id_2          100 non-null    object 
 13  name_2        100 non-null    object 
 14  latitude_2    100 non-null    float64
 15  longitude_2   100 non-null    float64
 16  address_2     52 non-null     object 
 17  city_2        68 non-null     object 
 18  state_2       60 non-null     o

True     73
False    27
Name: match, dtype: int64

In [19]:
word_columns = ['name_1','address_1','city_1','state_1','url_1','categories_1','name_2','address_2','city_2','state_2','url_2','categories_2']
url_columns = ['url_1','url_2']
pairs[url_columns] = remove_URL(pairs, url_columns)[url_columns]
pairs[word_columns] = lower(pairs, word_columns)[word_columns]
pairs[word_columns] = num_remove(pairs, word_columns)[word_columns]
pairs[word_columns] = punc_remove(pairs, word_columns)[word_columns]
pairs[word_columns] = space_remove(pairs, word_columns)[word_columns]
# pairs[word_columns] = stop_remove(pairs, word_columns)[word_columns]
# pairs[word_columns] = lemma_stemming(pairs, word_columns)[word_columns]

In [20]:
# wordnet similarity method
def similarity(word1, word2):
    max_sim = []
    syn1 = wn.synsets(word1)
    syn2 = wn.synsets(word2)
    for s1 in syn1:
        for s2 in syn2:
            max_sim.append(s1.path_similarity(s2))
    max_sim = [z for z in max_sim if z!=None]
    if max_sim == []:
        return 0
    return max(max_sim)
    
def similarity_lis(lis_1, lis_2):
    score_lis = []
    for word in lis_1:
        score_lis2 = []
        for word_2 in lis_2:
            sim = similarity(word, word_2)
            if sim != 0:
                score_lis2 += [sim]
        if score_lis2 == []:
            score_lis += [0]
        else:
            score_lis+=[max(score_lis2)]
    if score_lis == []:
        return 0
    return sum(score_lis)/len(score_lis)

def sim_lis(lis_1, lis_2):
    return (similarity_lis(lis_1, lis_2) + similarity_lis(lis_2, lis_1))/2

def nltk_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    for i in range(len(cols_1)):
        df[f"{cols_1[i]}_nltk"] = df.apply(lambda x: sim_lis(x[cols_1[i]],x[cols_2[i]]), axis = 1)
    return df  

In [21]:
col_1 = ['name_1','address_1','city_1','state_1','url_1','categories_1']
col_2 = ['name_2','address_2','city_2','state_2','url_2','categories_2']
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
pairs = nltk_similarity(pairs, col_1,col_2)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [22]:
pairs

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,url_2,phone_2,categories_2,match,name_1_nltk,address_1_nltk,city_1_nltk,state_1_nltk,url_1_nltk,categories_1_nltk
0,E_000001272c6c5d,café stad oudenaarde,50.859975,3.634196,abdijstraat,nederename,oostvlaanderen,9700,BE,,...,,NaN,bars,True,0.825000,0.000000,0.000000,0.000000,0.0,1.000000
1,E_000008a8ba4f48,turkcell,37.844510,27.844202,adnan menderes bulvarı,,,NaN,TR,,...,,NaN,electronics stores,False,1.000000,0.563636,0.000000,0.000000,0.0,0.694444
2,E_000023d8f4be44,island spa,14.518970,121.018702,th flr newport mall resorts world manila,pasay city,metro manila,NaN,PH,,...,,NaN,spas,True,0.783333,0.000000,0.000000,0.000000,0.0,1.000000
3,E_00007dcd2bb53f,togos sandwiches,38.257797,-122.064599,holiday ln ste b,fairfield,ca,94534,US,,...,,NaN,sandwich places,True,0.802083,0.000000,1.000000,1.000000,0.0,0.933333
4,E_0000c362229d93,coffee cat,7.082218,125.610244,f torres st,davao city,davao region,8000,PH,,...,,NaN,coffee shops cafés dessert shops,False,0.950000,0.607656,0.900000,0.916667,0.0,0.807292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,E_000d7da93f373c,ローソン 小樽色内店,43.198794,140.998369,色内,小樽市,北海道,047-0031,JP,,...,,NaN,convenience stores,True,0.000000,0.000000,0.000000,0.000000,0.0,0.944444
96,E_000da284c9870c,cityplace parade,43.640253,-79.397529,iceboat terrace dan leckie way,toronto,on,M5V 4A9,CA,,...,,NaN,residential buildings apartments condos,True,0.786806,0.810417,1.000000,1.000000,0.0,0.900000
97,E_000dfeb9eb4e87,plac zawiszy,52.225121,20.989843,,warszawa,mazowieckie,02-017,PL,,...,,NaN,bus stops,False,0.916667,0.000000,0.958333,0.723485,0.0,0.888889
98,E_000e3b02cbde30,walmart neighborhood market,36.055210,-115.119751,s eastern ave,las vegas,nv,89123,US,,...,,NaN,miscellaneous shops,True,0.827160,0.000000,0.888889,1.000000,0.0,0.795127


In [23]:
new_df = pairs[["match","name_1_nltk","address_1_nltk","city_1_nltk","state_1_nltk","url_1_nltk","categories_1_nltk"]]
new_df

,match,name_1_nltk,address_1_nltk,city_1_nltk,state_1_nltk,url_1_nltk,categories_1_nltk
0,True,0.825000,0.000000,0.000000,0.000000,0.0,1.000000
1,False,1.000000,0.563636,0.000000,0.000000,0.0,0.694444
2,True,0.783333,0.000000,0.000000,0.000000,0.0,1.000000
3,True,0.802083,0.000000,1.000000,1.000000,0.0,0.933333
4,False,0.950000,0.607656,0.900000,0.916667,0.0,0.807292
...,...,...,...,...,...,...,...
95,True,0.000000,0.000000,0.000000,0.000000,0.0,0.944444
96,True,0.786806,0.810417,1.000000,1.000000,0.0,0.900000
97,False,0.916667,0.000000,0.958333,0.723485,0.0,0.888889
98,True,0.827160,0.000000,0.888889,1.000000,0.0,0.795127


In [24]:
target_col = "match"
X = new_df.loc[:, new_df.columns != target_col]
y = new_df.loc[:, target_col]

In [66]:
y.value_counts()

True     73
False    27
Name: match, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [26]:
X_train.head
y_train.head

<bound method NDFrame.head of 28    False
93    False
5      True
66     True
65     True
      ...  
60    False
71     True
14    False
92    False
51     True
Name: match, Length: 67, dtype: bool>

In [27]:
model_cb = CatBoostClassifier(task_type='GPU', iterations=100, 
                              random_state = 2021, 
                              eval_metric="F1")

In [58]:
X_train


,name_1_nltk,address_1_nltk,city_1_nltk,state_1_nltk,url_1_nltk,categories_1_nltk
28,0.770833,0.774074,0.857143,0.333333,0.0,0.714286
93,0.427171,0.913043,0.909091,1.000000,0.0,0.809829
5,0.000000,0.000000,0.000000,0.000000,0.0,0.601810
66,0.900000,0.000000,0.000000,0.000000,0.0,0.944444
65,0.538889,0.928571,0.000000,0.000000,0.0,0.785714
...,...,...,...,...,...,...
60,0.434295,0.732639,1.000000,1.000000,0.0,0.598485
71,0.923077,0.529101,0.909091,0.941176,0.0,0.923077
14,0.000000,0.000000,0.000000,0.000000,0.0,0.928571
92,0.743728,0.000000,0.000000,0.000000,0.0,0.833333


In [59]:
model_cb.fit(X_train, y_train, plot=True, 
             eval_set=(X_test, y_test))
# cat_features= cat_features
model_cb.save_model('tiny_800_cb_2',format="cbm",
           export_parameters=None,
           pool=None)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.232439
0:	learn: 0.9052632	test: 0.7600000	best: 0.7600000 (0)	total: 32.6ms	remaining: 3.23s
1:	learn: 0.9000000	test: 0.7692308	best: 0.7692308 (1)	total: 63.7ms	remaining: 3.12s
2:	learn: 0.9038462	test: 0.8771930	best: 0.8771930 (2)	total: 93.9ms	remaining: 3.04s
3:	learn: 0.8952381	test: 0.8571429	best: 0.8771930 (2)	total: 117ms	remaining: 2.8s
4:	learn: 0.9126214	test: 0.8771930	best: 0.8771930 (2)	total: 121ms	remaining: 2.3s
5:	learn: 0.8952381	test: 0.8771930	best: 0.8771930 (2)	total: 126ms	remaining: 1.97s
6:	learn: 0.8867925	test: 0.8771930	best: 0.8771930 (2)	total: 131ms	remaining: 1.74s
7:	learn: 0.9126214	test: 0.8771930	best: 0.8771930 (2)	total: 135ms	remaining: 1.56s
8:	learn: 0.9038462	test: 0.8771930	best: 0.8771930 (2)	total: 144ms	remaining: 1.46s
9:	learn: 0.9126214	test: 0.8771930	best: 0.8771930 (2)	total: 149ms	remaining: 1.34s
10:	learn: 0.9038462	test: 0.8771930	best: 0.8771930 (2)	total: 154ms	remaining: 1.24s
11:	learn: 0.8952381	t

In [60]:
from_save = CatBoostClassifier()
model_cb= from_save.load_model("tiny_800_cb_2",format="cbm")


In [62]:
X_test

,name_1_nltk,address_1_nltk,city_1_nltk,state_1_nltk,url_1_nltk,categories_1_nltk
83,0.923077,1.000000,1.000000,0.000000,0.0,0.941176
53,0.900000,0.800000,1.000000,1.000000,0.0,1.000000
70,0.000000,0.000000,0.000000,0.000000,0.0,0.944444
45,0.712993,0.000000,0.000000,0.000000,0.0,0.597222
44,0.000000,0.000000,0.000000,0.000000,0.0,0.648810
39,0.000000,0.000000,0.000000,0.000000,0.0,0.886667
22,0.818627,0.000000,0.000000,0.000000,0.0,0.851541
80,0.792308,0.000000,0.000000,0.000000,0.0,0.705556
10,0.000000,0.000000,0.000000,0.000000,0.0,0.806250
0,0.825000,0.000000,0.000000,0.000000,0.0,1.000000


In [67]:
y_pred = model_cb.predict(X)


In [68]:
y_pred

array(['True', 'False', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'False', 'True', 'False', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'False', 'False', 'True', 'True',
       'True', 'True', 'True', 'True'], dtype=object)

In [51]:
y_pred

array(['True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True',
       'False'], dtype=object)

In [52]:
f1_score(y_test, y_pred)

TypeError: ignored

In [ ]:
accuracy_score(y_test, y_pred)